In [ ]:
## Restructured model with germany data

In [1]:
import pickle
from cntmosaic.dataloader.restru_loaders import MergedLoader, RawLoader
import pandas as pd
import numpy as np
import xarray as xr
from cntmosaic.dataloader import CoordToColumns
from cntmosaic.models.restr_BRCfine import restr_BRCfine

d:\Code\cntmosaic\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

with open('../datasets/data/polymod_germany.pkl', 'rb') as file:
    data = pickle.load(file)['contacts']


In [44]:
c = pd.read_csv('2008_Mossong_POLYMOD_contact_common.csv')
p = pd.read_csv('2008_Mossong_POLYMOD_participant_common.csv')


In [18]:
mapp = CoordToColumns(age_part='part_age', age_cnt='cnt_age_exact', id_var='part_id')
data = RawLoader(p, c, mapp)

In [19]:
data.raw_df = data.raw_df[data.raw_df.part_age < 85]
data.raw_df = data.raw_df[data.raw_df.cnt_age_exact < 85]

In [3]:
mapp = CoordToColumns(age_part='age_part', age_cnt='age_cnt', id_var='id', population_age='age', population_size='P')

In [4]:
with open('../datasets/data/polymod_germany.pkl', 'rb') as file:
    pop = pickle.load(file)['population']

In [5]:
d = MergedLoader(data, mapp, pop[pop.age < 85])

In [6]:
d.load()

In [7]:
model = restr_BRCfine(d)

new model instantiated, please check default hyperparameters


In [8]:
model.params.grid_type = 'diff-age'

In [9]:
import jax
model.run_inference_mcmc(jax.random.PRNGKey(0),
    num_samples = 10,
    num_warmup = 5,
    num_chains = 2)

D:\Code\cntmosaic\cntmosaic\models\_inference.py:28: UserWarning: There are not enough devices to run parallel chains: expected 2 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(2)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  mcmc = MCMC(
sample: 100%|██████████| 15/15 [00:01<00:00, 10.92it/s, 1 steps of size 4.32e-02. acc. prob=0.00]


Number of divergences: 17


In [60]:
# Save to a file
with open('diff-age.pkl', 'wb') as f:
    pickle.dump(model, f)


In [61]:
with open('diff-age.pkl', 'rb') as f:
    model = pickle.load(f)


In [10]:
from cntmosaic.analysis import ModelSummariserMCMC, ModelVisualiser

In [11]:
MSM = ModelSummariserMCMC(model)

In [64]:
visual = ModelVisualiser(MSM)

In [65]:
visual.plot_cint()

alt.Chart(...)

In [66]:
visual.plot_rate()

alt.Chart(...)

In [12]:
MSM.get_posterior()

In [14]:
MSM.post['log_rate'].shape

(20, 85, 85)

In [16]:
model._precompute.log_P.shape

(1, 85)

In [18]:
MSM.post['beta0']

Array([-0.36116496, -0.49339548, -1.090935  , -1.2619239 , -2.5391412 ,
       -3.0914845 , -2.7672985 , -2.8170865 , -2.8481107 , -2.7367206 ,
       -1.8352437 , -1.8352437 , -1.8352437 , -1.8352437 , -1.8352437 ,
       -1.8352437 , -1.8352437 , -1.8352437 , -1.8352437 , -1.8352437 ],      dtype=float32)